# TASK 1 : Install Dependencies & Run Spark Session

In [1]:
#install pyspark
! pip install pyspark  #command line script

In [2]:
#create a sparksession
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("spark").getOrCreate()


# TASK 2: Clone & Explore dataset

In [3]:
#clone the diabetes dataset from the github repository
! git clone https://github.com/education454/diabetes_dataset

fatal: destination path 'diabetes_dataset' already exists and is not an empty directory.


In [4]:
#check if the dataset exists
! ls diabetes_dataset

diabetes.csv  new_test.csv


In [5]:
#create spark dataframe
df = spark.read.csv("/content/diabetes_dataset/diabetes.csv",header =True, inferSchema=True)

In [6]:
#display the dataframe
df.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          2|    138|           62|           35|      0|33.6|                   0.127| 47|      1|
|          0|     84|           82|           31|    125|38.2|                   0.233| 23|      0|
|          0|    145|            0|            0|      0|44.2|                    0.63| 31|      1|
|          0|    135|           68|           42|    250|42.3|                   0.365| 24|      1|
|          1|    139|           62|           41|    480|40.7|                   0.536| 21|      0|
|          0|    173|           78|           32|    265|46.5|                   1.159| 58|      0|
|          4|     99|           72|           17|      0|25.6|                   0.294| 28|      0|


In [7]:
#print the schema
df.printSchema()

root
 |-- Pregnancies: integer (nullable = true)
 |-- Glucose: integer (nullable = true)
 |-- BloodPressure: integer (nullable = true)
 |-- SkinThickness: integer (nullable = true)
 |-- Insulin: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Outcome: integer (nullable = true)



In [8]:
#count the total no. of diabetic and non-diabetic class
print((df.count(),len(df.columns)))
df.groupBy('Outcome').count().show()

(2000, 9)
+-------+-----+
|Outcome|count|
+-------+-----+
|      1|  684|
|      0| 1316|
+-------+-----+



In [9]:
#get the summary statistics
df.describe().show()

+-------+-----------------+------------------+------------------+-----------------+-----------------+------------------+------------------------+------------------+------------------+
|summary|      Pregnancies|           Glucose|     BloodPressure|    SkinThickness|          Insulin|               BMI|DiabetesPedigreeFunction|               Age|           Outcome|
+-------+-----------------+------------------+------------------+-----------------+-----------------+------------------+------------------------+------------------+------------------+
|  count|             2000|              2000|              2000|             2000|             2000|              2000|                    2000|              2000|              2000|
|   mean|           3.7035|          121.1825|           69.1455|           20.935|           80.254|32.192999999999984|     0.47092999999999974|           33.0905|             0.342|
| stddev|3.306063032730656|32.068635649902916|19.188314815604098|16.103242909926

# TASK 3: Data Cleaning & Preparation

In [10]:
#check for null values
for col in df.columns:
  print(col+":",df[df[col].isNull()].count())

Pregnancies: 0
Glucose: 0
BloodPressure: 0
SkinThickness: 0
Insulin: 0
BMI: 0
DiabetesPedigreeFunction: 0
Age: 0
Outcome: 0


In [11]:
#look for the unnecessary values present
def count_zeros():
  columns_list=['Glucose','BloodPressure','SkinThickness','Insulin','BMI']
  for i in columns_list:
    print(i+":",df[df[i]==0].count())
count_zeros()

Glucose: 13
BloodPressure: 90
SkinThickness: 573
Insulin: 956
BMI: 28


In [12]:
#calculate and replace the unnecessary values by the mean value
from pyspark.sql.functions import *
for i in df.columns[1:6]:
  data=df.agg({i:'mean'}).first()[0]
  print("mean value for {} is {}".format(i,int(data)))
  df = df.withColumn(i,when(df[i]==0,int(data)).otherwise(df[i]))

mean value for Glucose is 121
mean value for BloodPressure is 69
mean value for SkinThickness is 20
mean value for Insulin is 80
mean value for BMI is 32


In [13]:
#display the dataframe
df.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          2|    138|           62|           35|     80|33.6|                   0.127| 47|      1|
|          0|     84|           82|           31|    125|38.2|                   0.233| 23|      0|
|          0|    145|           69|           20|     80|44.2|                    0.63| 31|      1|
|          0|    135|           68|           42|    250|42.3|                   0.365| 24|      1|
|          1|    139|           62|           41|    480|40.7|                   0.536| 21|      0|
|          0|    173|           78|           32|    265|46.5|                   1.159| 58|      0|
|          4|     99|           72|           17|     80|25.6|                   0.294| 28|      0|


# TASK 4: Correlation Analysis & Feature Selection

In [14]:
#find the correlation among the set of input & output variables
for i in df.columns:
  print("Correlation to Outcome for {} is {}".format(i,df.stat.corr('Outcome',i)))

Correlation to Outcome for Pregnancies is 0.22443699263363961
Correlation to Outcome for Glucose is 0.48796646527321064
Correlation to Outcome for BloodPressure is 0.17171333286446713
Correlation to Outcome for SkinThickness is 0.1659010662889893
Correlation to Outcome for Insulin is 0.1711763270226193
Correlation to Outcome for BMI is 0.2827927569760082
Correlation to Outcome for DiabetesPedigreeFunction is 0.1554590791569403
Correlation to Outcome for Age is 0.23650924717620253
Correlation to Outcome for Outcome is 1.0


In [15]:
#feature selection
from pyspark.ml.feature import VectorAssembler
assembler= VectorAssembler(inputCols=['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin','BMI','DiabetesPedigreeFunction','Age'],outputCol='features')
output_data = assembler.transform(df)

In [16]:
#print the schema
output_data.printSchema()

root
 |-- Pregnancies: integer (nullable = true)
 |-- Glucose: integer (nullable = true)
 |-- BloodPressure: integer (nullable = true)
 |-- SkinThickness: integer (nullable = true)
 |-- Insulin: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Outcome: integer (nullable = true)
 |-- features: vector (nullable = true)



In [17]:
#display dataframe
output_data.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+--------------------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|            features|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+--------------------+
|          2|    138|           62|           35|     80|33.6|                   0.127| 47|      1|[2.0,138.0,62.0,3...|
|          0|     84|           82|           31|    125|38.2|                   0.233| 23|      0|[0.0,84.0,82.0,31...|
|          0|    145|           69|           20|     80|44.2|                    0.63| 31|      1|[0.0,145.0,69.0,2...|
|          0|    135|           68|           42|    250|42.3|                   0.365| 24|      1|[0.0,135.0,68.0,4...|
|          1|    139|           62|           41|    480|40.7|                   0.536| 21|      0|[1.0,139.0,62.0,4...|
|          0|    173|           

# TASK 5: Split Dataset & Build the Model

In [18]:
#create final data
from pyspark.ml.classification import LogisticRegression
final_data=output_data.select('features','Outcome')

In [19]:
#print schema of final data
final_data.printSchema()

root
 |-- features: vector (nullable = true)
 |-- Outcome: integer (nullable = true)



In [20]:
#split the dataset ; build the model
train , test=final_data.randomSplit([0.7,0.3])
models = LogisticRegression(labelCol='Outcome')
model=models.fit(train)

In [21]:
#summary of the model
summary=model.summary
summary.predictions.describe().show()

+-------+------------------+------------------+
|summary|           Outcome|        prediction|
+-------+------------------+------------------+
|  count|              1423|              1423|
|   mean|0.3443429374560787|0.2656359803232607|
| stddev|0.4753205748094818|0.4418265369744437|
|    min|               0.0|               0.0|
|    max|               1.0|               1.0|
+-------+------------------+------------------+



# TASK 6: Evaluate and Save the Model

In [22]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
predictions=model.evaluate(test)

In [23]:
predictions.predictions.show()

+--------------------+-------+--------------------+--------------------+----------+
|            features|Outcome|       rawPrediction|         probability|prediction|
+--------------------+-------+--------------------+--------------------+----------+
|[0.0,57.0,60.0,20...|      0|[4.19070552322845...|[0.98509006821607...|       0.0|
|[0.0,57.0,60.0,20...|      0|[4.19070552322845...|[0.98509006821607...|       0.0|
|[0.0,67.0,76.0,20...|      0|[2.78016399561360...|[0.94159446393956...|       0.0|
|[0.0,67.0,76.0,20...|      0|[2.78016399561360...|[0.94159446393956...|       0.0|
|[0.0,73.0,69.0,20...|      0|[4.33035734320296...|[0.98700816646793...|       0.0|
|[0.0,74.0,52.0,10...|      0|[3.90589311191166...|[0.98027397196189...|       0.0|
|[0.0,78.0,88.0,29...|      0|[2.87733091212096...|[0.94671437851917...|       0.0|
|[0.0,84.0,64.0,22...|      0|[2.64833787379047...|[0.93390847232613...|       0.0|
|[0.0,84.0,64.0,22...|      0|[2.64833787379047...|[0.93390847232613...|    

In [24]:
evaluator=BinaryClassificationEvaluator(rawPredictionCol= 'rawPrediction',labelCol='Outcome')
evaluator.evaluate(model.transform(test))

0.819654921805604

In [25]:
# save model
model.save("model")

Py4JJavaError: An error occurred while calling o358.save.
: java.io.IOException: Path model already exists. To overwrite it, please use write.overwrite().save(path) for Scala and use write().overwrite().save(path) for Java and Python.
	at org.apache.spark.ml.util.FileSystemOverwrite.handleOverwrite(ReadWrite.scala:683)
	at org.apache.spark.ml.util.MLWriter.save(ReadWrite.scala:167)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:566)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j

In [26]:
# load saved model back to the environment
from pyspark.ml.classification import LogisticRegressionModel
model=LogisticRegressionModel.load('model')

# TASK 7: Prediction on New Data with the saved model

In [27]:
#create a new spark dataframe
test_df=spark.read.csv("/content/new_test.csv",header=True,inferSchema=True)

In [28]:
#print the schema
test_df.printSchema()

root
 |-- Pregnancies: integer (nullable = true)
 |-- Glucose: integer (nullable = true)
 |-- BloodPressure: integer (nullable = true)
 |-- SkinThickness: integer (nullable = true)
 |-- Insulin: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: integer (nullable = true)



In [29]:
#create an additional feature merged column
test_data=assembler.transform(test_df)

In [30]:
#print the schema
test_data.printSchema()

root
 |-- Pregnancies: integer (nullable = true)
 |-- Glucose: integer (nullable = true)
 |-- BloodPressure: integer (nullable = true)
 |-- SkinThickness: integer (nullable = true)
 |-- Insulin: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: integer (nullable = true)
 |-- features: vector (nullable = true)



In [31]:
#use model to make predictions
results=model.transform(test_data)
results.printSchema()

root
 |-- Pregnancies: integer (nullable = true)
 |-- Glucose: integer (nullable = true)
 |-- BloodPressure: integer (nullable = true)
 |-- SkinThickness: integer (nullable = true)
 |-- Insulin: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: integer (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [32]:
#display the predictions
results.select('features','prediction').show()

+--------------------+----------+
|            features|prediction|
+--------------------+----------+
|[1.0,190.0,78.0,3...|       1.0|
|[0.0,80.0,84.0,36...|       0.0|
|[2.0,138.0,82.0,4...|       1.0|
|[1.0,110.0,63.0,4...|       1.0|
+--------------------+----------+

